# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
print(y.head())

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64


In [ ]:
# Define features set X by selecting all columns but credit_ranking

X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
print(X.head())

   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset


# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Now transform (scale) both the training and testing datasets
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model

number_of_features = X_train_scaled.shape[1]
# Review the number of features
print("Number of features:", number_of_features)

Number of features: 11


In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 10

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5

# Define the number of neurons in the output layer
output_neurons = 1

In [ ]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, input_dim=number_of_features, activation='relu'))


# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))


In [ ]:
# Display the Sequential model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                120       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 181 (724.00 Byte)
Trainable params: 181 (724.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
32/32 [==============================] - 2s 9ms/step - loss: 0.6736 - accuracy: 0.6158 - val_loss: 0.6559 - val_accuracy: 0.6641
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 0.6477 - accuracy: 0.6686 - val_loss: 0.6389 - val_accuracy: 0.6875
Epoch 3/50
32/32 [==============================] - 0s 3ms/step - loss: 0.6273 - accuracy: 0.6940 - val_loss: 0.6228 - val_accuracy: 0.6992
Epoch 4/50
32/32 [==============================] - 0s 3ms/step - loss: 0.6089 - accuracy: 0.7048 - val_loss: 0.6056 - val_accuracy: 0.7031
Epoch 5/50
32/32 [==============================] - 0s 3ms/step - loss: 0.5887 - accuracy: 0.7185 - val_loss: 0.5885 - val_accuracy: 0.6953
Epoch 6/50
32/32 [==============================] - 0s 3ms/step - loss: 0.5698 - accuracy: 0.7214 - val_loss: 0.5729 - val_accuracy: 0.7070
Epoch 7/50
32/32 [==============================] - 0s 3ms/step - loss: 0.5534 - accuracy: 0.7283 - val_loss: 0.5589 - val_accuracy: 0.7227
Epoch 8/50
32/32 [==

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)


# Display the model loss and accuracy results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

10/10 [==============================] - 0s 4ms/step - loss: 0.5066 - accuracy: 0.7469
Test Loss: 0.506565272808075
Test Accuracy: 0.746874988079071


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
model_file_path = '/content/drive/My Drive/student_loans.keras'


# Export your model to a keras file
model.save(model_file_path)
print(f'Model saved to {model_file_path}')


Model saved to /content/drive/My Drive/student_loans.keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
model_file_path = '/content/drive/My Drive/student_loans.keras'

# Load the model to a new object
from tensorflow.keras.models import load_model

loaded_model = load_model(model_file_path)

# Display summary of the loaded model
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                120       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 181 (724.00 Byte)
Trainable params: 181 (724.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled, verbose=2)


# Display a sample of the predictions
print(predictions[:5])


10/10 - 0s - 161ms/epoch - 16ms/step
[[0.43142354]
 [0.28323743]
 [0.6922134 ]
 [0.73701656]
 [0.9700988 ]]


In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Convert predictions to binary results (0 or 1) based on a threshold of 0.5
binary_predictions = (predictions > 0.5).astype(float)

# Save the binary predictions to a DataFrame
predictions_df = pd.DataFrame(binary_predictions, columns=['predictions'])

# Display the DataFrame
print(predictions_df)

     predictions
0            0.0
1            0.0
2            1.0
3            1.0
4            1.0
..           ...
315          0.0
316          0.0
317          0.0
318          1.0
319          1.0

[320 rows x 1 columns]


### Step 4: Display a classification report with the y

1.   List item
2.   List item

test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, binary_predictions.flatten()))


              precision    recall  f1-score   support

           0       0.72      0.77      0.75       154
           1       0.77      0.72      0.75       166

    accuracy                           0.75       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.75      0.75      0.75       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

### Data Collection for a Student Loan Recommendation System

To build an effective recommendation system for student loan options, the following data would be relevant:

1. **Student Demographics**: Age, location, field of study, and level of education. This data could help tailor loan options that are available and suitable for students in specific demographics.
2. **Financial Information**: Income level (if applicable), dependency status, and financial need. Understanding a student's financial situation is crucial for recommending loans that they are eligible for and can afford.
3. **Educational Institution Details**: Cost of attendance, public vs. private status, and graduation rates. Different institutions have varying costs, which affect the amount of loan needed.
4. **Existing Loans and Grants**: Details on any existing student loans or grants, including amounts and terms. This helps in understanding the additional financial assistance needed.
5. **Career Goals and Prospects**: Expected graduation date, career aspirations, and potential starting salary. Loans with repayment plans that consider future income could be more appropriate for some students.

This data is relevant because it encompasses both the financial need and the capacity to repay, ensuring that recommended loan options are both accessible to the student and financially responsible.

### Filtering Method for the Recommendation System

Given the data types described, **content-based filtering** would be a suitable approach for this recommendation system. This method recommends items based on a comparison between the content of the items and a user profile. In this context, the "content" includes the loan terms, eligibility criteria, and features (such as interest rates, repayment options), while the user profile would consist of the student's financial situation, demographics, and educational details.

Content-based filtering is suitable here because it allows for personalized loan recommendations tailored to the individual's specific needs and circumstances, without necessarily requiring data on other users' preferences or choices, which is a key aspect of collaborative filtering.

### Real-world Challenges

1. **Data Privacy and Security**: Financial and personal data are highly sensitive. Ensuring the privacy and security of this data is paramount. Challenges include safeguarding against data breaches and complying with regulations like GDPR and FERPA. This concern is crucial because any compromise of student data could have severe financial and personal repercussions.

2. **Bias and Fairness**: Ensuring the recommendation system doesn't inherit or amplify biases present in historical data is essential. For example, loan options should not inadvertently favor students from certain demographics or financial backgrounds. This is of concern because biases in recommendations could perpetuate inequalities in access to education or lead to financial recommendations that aren't in the best interest of all students.

Addressing these challenges is essential for the development of a recommendation system that is not only effective but also ethical and equitable, ensuring that all students receive appropriate and fair loan recommendations.